<a href="https://colab.research.google.com/github/niphadkarneha/FaceRecognition/blob/master/ImageRecognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

There are 2 parts to this tutorial, where we aim to detect faces in a group of faces.

Part 1: We will detect one image in a group.
Part 2: We will detect two images in a group and label the others as unknown.

In order to follow through, you will need to carry out certain set up steps

1.   Change Runtime type
> On the Menu bar, click on Runtime | 
Click on, "Change Runtime Type" | 
Select, "Hardware Accelerator" as GPU | 

2.   Install the necessary libraries.

In [0]:
# Install face_recognition library if it has not been installed.

pip install face_recognition

In [0]:
# Import face_recognition, NumPy, and Pillow libraries

import face_recognition
import numpy as np
from PIL import Image, ImageDraw

In [0]:
# I have mounted my Google Drive Folder which contains the images I want to train and test. This folder will also save the resulting image.
# You need to create a folder in your Google Drive, I have named it IR which stands for, "Image Recognition". In order to make sure, we have a successfull connection
# I am going to create a text file, write, "Successfully mounted!" in it and display the message. This is a sanity check that our drive was mounted correctly.

from google.colab import drive
drive.mount('/content/drive')
with open('/content/drive/My Drive/IR/foo.txt', 'w') as f:
  f.write('Successfully mounted!')
!cat /content/drive/My\ Drive/IR/foo.txt

In [0]:
# Next, we will load the known images by giving the complete path of the image.

known_image = face_recognition.load_image_file("/content/drive/My Drive/IR/profile.jpg")
encoding = face_recognition.face_encodings(known_image)[0]

In [0]:
# We'll load an unknown image or a test image, again make sure you are giving the complete path.

unknown_image = face_recognition.load_image_file("/content/drive/My Drive/IR/neha.jpg")

In [0]:
# Pillow==5.3.0 was needed to overcome a Type Error.

pip install Pillow==5.3.0

In [0]:
# Find all the faces and face encodings in the unknown image
face_locations = face_recognition.face_locations(unknown_image)
face_encodings = face_recognition.face_encodings(unknown_image, face_locations)

# Convert the image to a PIL-format image so that we can draw on top of it with the Pillow library
# See http://pillow.readthedocs.io/ for more about PIL/Pillow
pil_image = Image.fromarray(unknown_image)

# Create a Pillow ImageDraw Draw instance to draw with
draw = ImageDraw.Draw(pil_image)

# Loop through each face found in the unknown image
for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):

    # See if the face is a match for the known face(s)
    matches = face_recognition.compare_faces([encoding], face_encoding)

    # Use the known face with the smallest distance to the new face
    face_distances = face_recognition.face_distance([encoding], face_encoding)
    best_match_index = np.argmin(face_distances)
    if matches[best_match_index]:

        # Draw a box around the face using the Pillow module
        draw.rectangle(((left - 20, top - 20), (right + 20, bottom + 20)), outline=(0, 255, 0), width=20)

# Remove the drawing library from memory as per the Pillow docs
del draw

# Saving the resulting image
pil_image.save("/content/drive/My Drive/IR/result.jpg")

In [0]:
# Importing the IPython library to display the before and after images on the console.

from IPython.display import Image, display
display(Image('/content/drive/My Drive/IR/profile.jpg'))
display(Image('/content/drive/My Drive/IR/result.jpg',width=500, height=500))

In [0]:
# Second Part of the Tutorial, where we would be detecting mine and Dr. Ravi's photo in a group photo
# and tag every other person's photo as unknown.

neha_image = face_recognition.load_image_file("/content/drive/My Drive/IR/profile.jpg")
neha_face_encoding = face_recognition.face_encodings(neha_image)[0]

# Load a second sample picture and learn how to recognize it.
ravim_image = face_recognition.load_image_file("/content/drive/My Drive/IR/ravi.png")
ravim_face_encoding = face_recognition.face_encodings(ravim_image)[0]

# Create arrays of known face encodings and their names
known_face_encodings = [
    neha_face_encoding,
    ravim_face_encoding
]
known_face_names = [
    "Neha Niphadkar",
    "Ravi Mukkamala. Dept.Chair"
]

# Load an image with an unknown face
unknown_image = face_recognition.load_image_file("/content/drive/My Drive/IR/neha.jpg")

# Find all the faces and face encodings in the unknown image
face_locations = face_recognition.face_locations(unknown_image)
face_encodings = face_recognition.face_encodings(unknown_image, face_locations)

# Convert the image to a PIL-format image so that we can draw on top of it with the Pillow library
# See http://pillow.readthedocs.io/ for more about PIL/Pillow
pil_image = Image.fromarray(unknown_image)
# Create a Pillow ImageDraw Draw instance to draw with
draw = ImageDraw.Draw(pil_image)

# Loop through each face found in the unknown image
for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
    # See if the face is a match for the known face(s)
    matches = face_recognition.compare_faces(known_face_encodings, face_encoding)

    name = "Unknown"

    # If a match was found in known_face_encodings, just use the first one.
    # if True in matches:
    #     first_match_index = matches.index(True)
    #     name = known_face_names[first_match_index]

    # Or instead, use the known face with the smallest distance to the new face
    face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
    best_match_index = np.argmin(face_distances)
    if matches[best_match_index]:
        name = known_face_names[best_match_index]

    # Draw a box around the face using the Pillow module
    draw.rectangle(((left, top), (right, bottom)), outline=(0, 0, 255))

    # Draw a label with a name below the face
    text_width, text_height = draw.textsize(name)
    draw.rectangle(((left, bottom - text_height - 10), (right, bottom)), fill=(0, 0, 255), outline=(0, 0, 255))
    draw.text((left + 6, bottom - text_height - 5), name, fill=(255, 255, 255, 255))


# Remove the drawing library from memory as per the Pillow docs
del draw

# Display the resulting image
pil_image.save("/content/drive/My Drive/IR/result_neha_ravim.jpg")

In [0]:
# Importing the IPython library to display the before and after images on the console.

from IPython.display import Image, display
display(Image('/content/drive/My Drive/IR/profile.jpg'))
display(Image('/content/drive/My Drive/IR/ravi.png'))
display(Image('/content/drive/My Drive/IR/result_neha_ravim.jpg',width=500, height=500))